<a href="https://colab.research.google.com/github/LouisVanLangendonck/UPC-AML-ArchitectureClassif/blob/main/data_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Scraper for Architectural Stlye Images

This image web scraper is written in function of using it in Google Colab. Use outside of this context will need small adaptations to the code. 

Start by defining where the Architecture Classification MAIN DIRECTORY is located (the data directory substructure will automatically be created, and images downloaded in accordingly)

In [ ]:
project_dir = '/content/drive/MyDrive/AML'

Lets create the directory substructure...

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

try:
  os.makedirs(project_dir+'/data/unzipped/all_data/')
except:
  pass

data_dir = project_dir + '/data/unzipped/all_data/'
style_list = ['baroque', 'contemporary', 'gothic', 'modernism', 'neoclassicism', 'noucentisme', 'renaissance', 'romanesque']

for style in style_list:
  try:
    os.mkdir(os.path.join(data_dir, style))
  except:
    pass

Mounted at /content/drive


Necessary Webdriver installs and package parametrisation in order to run Selenium on Google Colab.. 

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

The actual functions that manouver through, download and correctly place the different images of all buildings of a respective style to be found on 'http://invarquit.cultura.gencat.cat/Cerca/'. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time 
import requests
from bs4 import BeautifulSoup
import urllib.request

driver = wd

def scrape_style(url, style_name):
    big_iter = 0
    base = 'http://invarquit.cultura.gencat.cat/Cerca/'  
    driver.get(url)
    first_building = driver.find_elements(By.CLASS_NAME, "enlacedestacado")[0] #Do this only once. From here on uses other button
    first_building.click()
    download_images(driver.current_url, style=style_name, big_it=big_iter) #To click trough all images and download each for first 
    next_buttons = driver.find_elements(By.TAG_NAME, 'img') 
    for button in next_buttons:
        if 'endavant' in button.get_attribute('src'):
            if "public" in button.get_attribute('title'):
                next_building = button
    next_building.click() #Go to next building
    contin=True
    while contin:
        big_iter += 1
        next_buttons = driver.find_elements(By.TAG_NAME, 'img')
        for button in next_buttons:
            if 'endavant' in button.get_attribute('src'):
                if "public" in button.get_attribute('title') and not "Off" in button.get_attribute('title'):
                    next_building = button #There is a next building 
                elif "public" in button.get_attribute('title') and "Off" in button.get_attribute('title'): #Last building of style
                    download_images(driver.current_url, style=style_name, big_it=big_iter) #To click trough all images and download each 
                    contin = False   
        if contin:
            next_building.click()
            download_images(driver.current_url, style=style_name, big_it=big_iter) #To click trough all images and download each 


def download_images(url, style, big_it):
    it = 0
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        candid_dupl = soup.find_all(class_ = "taulesDeDins") 
        if len(candid_dupl[0]) == 5:                    #To check if only a single style label is assigned to the building
            img_source = driver.find_element(By.ID, "imatgeACanviar").get_attribute("src")
            urllib.request.urlretrieve(img_source, '{}/{}/{}-{}-{}.jpg'.format(data_dir, style,style, big_it, it)) #Naming convention
            next_buttons = driver.find_elements(By.TAG_NAME, 'img')
            for button in next_buttons:
                if 'endavant' in button.get_attribute('src'):
                    if not "public" in button.get_attribute('title'):
                        next_img = button
            next_img.click()
            new_img_source = driver.find_element(By.ID, "imatgeACanviar").get_attribute("src")
            while new_img_source != img_source and it < 7: #Download at most 7 images
                it += 1
                urllib.request.urlretrieve(new_img_source, '{}/{}/{}-{}-{}.jpg'.format(data_dir,style,style, big_it, it))
                next_buttons = driver.find_elements(By.TAG_NAME, 'img')
                for button in next_buttons:
                    if 'endavant' in button.get_attribute('src'):
                        if not "public" in button.get_attribute('title'):
                            next_img = button
                next_img.click()
                img_source = new_img_source
                new_img_source = driver.find_element(By.ID, "imatgeACanviar").get_attribute("src")   
    except:
        pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Here the function is iteratively called for the styles specified. Note that it takes a very long time to run the whole loop... Check the files in the data file to see if the images are actually being loaded in! 

In [ ]:
style_list = ['baroque', 'contemporary', 'gothic', 'modernism', 'neoclassicism', 'noucentisme', 'renaissance', 'romanesque']
corr_url = ['http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K0JhcnJvYyU%3D', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K0RhcnJlcmVzIHRlbmTDqG5jaWVzJQ%3D%3D',
'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K0fDsnRpYyU%3D', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K01vZGVybmlzbWUl', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K05lb2NsYXNzaWNpc21lJQ%3D%3D',
'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K05vdWNlbnRpc21lJQ%3D%3D', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K1JlbmFpeGVtZW50JQ%3D%3D', 
'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K1JvbcOgbmljJQ%3D%3D'] #Loop of all links necessary. Can easily be extended.

for idx, style in enumerate(style_list): #Actual command to run and scrape all. Note that everything will take a long time. 
    URL = corr_url[idx]
    scrape_style(URL, style)